# Model Developement, Treaning and Evaluation 

## Step 1: Setting up the notebook

In [1]:

# importing the necossery liberay 

import sagemaker
from sagemaker import get_execution_role


import os
import boto3
import numpy as np
import pandas as pd
import tensorflow as tf


from numpy import argmax
from pickle import load

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from nltk.translate.bleu_score import corpus_bleu


Using TensorFlow backend.


In [2]:
# This is an object that represents the SageMaker session that we are currently operating in. This
# object contains some useful information that we will need to access later such as our region.
Sagemaker_session = sagemaker.Session()

# This is an object that represents the IAM role that we are currently assigned. When we construct
# and launch the training job later we will need to tell it what IAM role it should have. Since our
# use case is relatively simple we will simply assign the training job the role we currently have.
role = get_execution_role()

In [3]:
%pwd

'/home/ec2-user/SageMaker/Modeling '

In [4]:
train_dir = "/home/ec2-user/SageMaker/Data Preprocessing/Dataset/train"
test_dir  = "/home/ec2-user/SageMaker/Data Preprocessing/Dataset/test"

## Step 2: Uploading the data files to S3

### Preprocessed data files

In [5]:
# Upload the preprossed data to this sagemaker instance's s3 

s3_prefix = 'imagecaptioning_vivek' 

traindata_s3_prefix = '{}/data/train'.format(s3_prefix)
testdata_s3_prefix = '{}/data/test'.format(s3_prefix)

train_s3 = Sagemaker_session.upload_data(path= train_dir, key_prefix=traindata_s3_prefix)
test_s3 = Sagemaker_session.upload_data(path=  test_dir, key_prefix=testdata_s3_prefix)



### Creating Inputs 

In [6]:
inputs = {
            'train':train_s3,
            'test': test_s3,
         }   # locations of the data in the S3

In [7]:
inputs

{'train': 's3://sagemaker-us-east-1-478270364551/imagecaptioning_vivek/data/train',
 'test': 's3://sagemaker-us-east-1-478270364551/imagecaptioning_vivek/data/test'}

## Step 3: Model Developement

In [8]:
!pygmentize 'Train/train.py'

import argparse
import os
import numpy as np
import json


## Get the model 

from model import get_model 

import tensorflow as tf



def _load_training_data(train_dir):
    """ loading training data from S3 """

    X1train = np.load(os.path.join(train_dir, 'X1train.npz'))['arr_0']
    X2train = np.load(os.path.join(train_dir, 'X2train.npz'))['arr_0']

    ytrain = np.load(os.path.join(train_dir, 'ytrain.npz'))['arr_0']

    return X1train,  X2train , ytrain

def _load_testing_data(test_dir):
    """ loading testing data from S3 """

    X1test = np.load(os.path.join(test_dir, 'X1test.npz'))['arr_0']
    X2test = np.load(os.path.join(test_dir, 'X2test.npz'))['arr_0']
    ytest = np.load(os.path.join(test_dir, 'ytest.npz'))['arr_0']

    return X1test,  X2test , ytest


def _parse_args():
    
    """ Parsing the arguments """
    parser = argparse.ArgumentParser()

    # Data, model, and output directories
    # model_dir is always passed in from SageMaker. By default, this is an S3 

In [9]:
!pygmentize 'Train/model.py'

# importing neccossory liberies

import tensorflow as tf


def get_model(input_im_feature_shape = 1000  , vocab_size = 7579 , max_length = 34 ):
    
    
    """
    creates an encoder decocer  arcitechure

    Parameters:

    input_im_feature_shape 
    ==> The shape of the feature vector 
    of the images extrated using Transfer learniong model 

    vocab_size ==> size of the vocabolary ( determined during data preprocessing )

    max_length ==> maximum length of input sequence 

    Returns:
    keras.models.Model object 

    """

    # feature extractor model

    inputs1 = tf.keras.layers.Input(shape=(input_im_feature_shape,))
    
    fe1 = tf.keras.layers.Dense(256, activation='relu')(inputs1)
    fe2 = tf.keras.layers.Dropout(0.3)(fe1)


    # sequence model
    inputs2 = tf.keras.layers.Input(shape=(max_length,))
    
    se1 = tf.keras.layers.Embedding(vocab_size, 256, mask_zero=True)(inputs2)
    se2 = tf.keras.layers.Dropout(0.3)(se1)


    #merge layer
    merge = tf

In [12]:
from Train.train import get_model

model = get_model()

In [13]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1000)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 34)]         0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 256)          256256      input_1[0][0]                    
__________________________________________________________________________________________________
embedding (Embedding)           (None, 34, 256)      1940224     input_2[0][0]                    
______________________________________________________________________________________________

## Step 4: Model Treaning  

In [10]:

from sagemaker.tensorflow import TensorFlow


estimator = TensorFlow(entry_point='train.py',
                     source_dir="Train",
                     role=role,
                     train_instance_count=1,
                     train_instance_type='ml.p2.xlarge',
                     framework_version='2.1.0',
                     py_version='py3',
                     distributions={'parameter_server': {'enabled': True}})

distributions has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [11]:
estimator.fit(inputs)

2021-02-28 13:12:34 Starting - Starting the training job...
2021-02-28 13:12:58 Starting - Launching requested ML instancesProfilerReport-1614517954: InProgress
......
2021-02-28 13:13:59 Starting - Preparing the instances for training.........
2021-02-28 13:15:19 Downloading - Downloading input data...
2021-02-28 13:16:00 Training - Downloading the training image......
2021-02-28 13:17:01 Training - Training image download completed. Training in progress..2021-02-28 13:17:06,370 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2021-02-28 13:17:06,893 sagemaker-containers INFO     Invoking user script

Training Env:

{
    "additional_framework_parameters": {
        "sagemaker_parameter_server_enabled": true
    },
    "channel_input_dirs": {
        "test": "/opt/ml/input/data/test",
        "train": "/opt/ml/input/data/train"
    },
    "current_host": "algo-1",
    "framework_module": "sagemaker_tensorflow_container.training:main",
    "hosts

In [12]:
model_data=estimator.model_data

In [13]:
model_data

's3://sagemaker-us-east-1-478270364551/tensorflow-training-2021-02-28-13-12-34-041/output/model.tar.gz'

## Step 5: Testing 

### We will first deploy the model and use the model for the testing 

NOTE : Its different from what we will be doing in the production level deployment ( we will create a sagemaker.tensorflow.TensorFlowModel object 

In [37]:
# deploying th eendpoint 

predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


-------------!

In [45]:

tokenizer = load(open('/home/ec2-user/SageMaker/Data Preprocessing/Dataset/tokenizer.pkl' ,'rb'))

test_features = load(open('/home/ec2-user/SageMaker/Data Preprocessing/Dataset/test_features' ,'rb'))

test_descriptions = load(open('/home/ec2-user/SageMaker/Data Preprocessing/Dataset/test_descriptions' ,'rb'))


In [46]:

def generate_desc( tokenizer, feature , max_length = 34 ):
    
    """ funcion to generate a description for an image"""
    
    in_text = 'startseq' # to seed the generation process
    
     
    for i in range(max_length):
        
        # integer encode and pad input sequence 
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        sequence = pad_sequences([sequence], maxlen=max_length)
    
        # predict next word
        inputs = {
                  "input_1": feature.tolist() , 
                    "input_2": sequence[0].tolist()
                    }

        result = predictor.predict(inputs)
        
        yhat = np.argmax(result['predictions'])
 
        # map integer to word
        word = word_for_id(yhat, tokenizer)
        
        # stop if we cannot map the word
        if word is None:
            break
        
        in_text += ' ' + word # append the input for generating the next word
        
        # stop if we predict the end of the sequence
        if word == 'endseq':
            break
            
            
    return in_text


def word_for_id(integer, tokenizer):
    
    """ function to map an integer to a word"""
    
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
        
    return None

In [56]:

actual, predicted = list(), list()


def evaluate_model(descriptions, features, tokenizer, max_length = 34 ):
    
    """Function to  evaluate the skill of the model"""
    
    # step over the whole set
    for key, desc_list in descriptions.items():
        
        #print(features[key][0].shape)
        
        # generate description
        yhat = generate_desc(tokenizer, features[key][0])
        
        # store actual and predicted
        references = [d.split() for d in desc_list]
        
        actual.append(references)
        
        predicted.append(yhat.split())
        
    # calculate BLEU score
    print('BLEU: %f' % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))


In [57]:
evaluate_model(test_descriptions, test_features, tokenizer, max_length = 34 )

BLEU: 0.460204


In [58]:
pd.DataFrame({"Actual" : actual, "Predicted" : predicted}).head(10)

,Actual,Predicted
0,"[[startseq, child, and, woman, are, at, waters...","[startseq, man, in, the, water, at, the, end, ..."
1,"[[startseq, boy, with, stick, kneeling, in, fr...","[startseq, two, children, are, playing, in, th..."
2,"[[startseq, woman, crouches, near, three, dogs...","[startseq, two, dogs, are, playing, in, the, s..."
3,"[[startseq, boy, bites, hard, into, treat, whi...","[startseq, young, boy, in, blue, shirt, and, b..."
4,"[[startseq, person, eats, takeout, while, watc...","[startseq, man, in, black, shirt, and, black, ..."
5,"[[startseq, the, children, are, playing, in, t...","[startseq, two, dogs, are, running, on, the, b..."
6,"[[startseq, group, of, eight, people, are, gat...","[startseq, man, in, black, and, white, tshirt,..."
7,"[[startseq, boy, in, blue, shorts, slides, dow...","[startseq, young, boy, in, blue, shirt, and, b..."
8,"[[startseq, man, in, wetsuit, is, throwing, ba...","[startseq, man, in, red, shirt, and, black, an..."
9,"[[startseq, family, playing, on, tractor, on, ...","[startseq, man, in, blue, shirt, and, blue, je..."


## Step 6: Shutting Down The endpoint 

In [ ]:
predictor.delete_endpoint()